# PoopPower: Differential abundance analysis

In the previous exercise we saw how to compare samples using alpha and beta diversity metrics. Now we will see with  _differential abundance_ how to test whether individual ASVs/taxa differ in abundance between samples' groups.
We are going to use the trustfull test  ANCOM : a compositionally aware alternative that allows to test for differentially abundant features. 

In [7]:
# importing all required packages & notebook extensions at the start of the notebook
import os
import matplotlib.pyplot as plt
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization
import seaborn as sns
from scipy.stats import shapiro, kruskal, f_oneway

In [8]:
div_dir = 'poop_data/Diversity'
phy_dir = 'poop_data/Phylogeny'
tox_dir = 'poop_data/Taxonomy'
den_dir= 'poop_data/Denoising'
diff_abu= 'poop_data/Differential_abundance'
data_dir = 'poop_data'

%matplotlib inline

##  ANCOM

Analyze this dataset given its limitations: 
1.We will start by filtering our feature table and only retain features that are present at some minimal frequency (5000). This can improve resolution and limit the false discovery rate penalty on features that are too far below the noise threshhold to be applicable to a statistical test. We can use the `filter-features` action from the `feature-table` plugin. 

In [33]:
! qiime feature-table filter-features \
    --i-table $den_dir/dada2_table.qza \
    --p-min-frequency 5000 \
    --o-filtered-table $diff_abu/table_abund.qza

Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund.qza


In [24]:
! qiime feature-table filter-features --help

Usage: qiime feature-table filter-features [OPTIONS]

  Filter features from table based on frequency and/or metadata. Any samples
  with a frequency of zero after feature filtering will also be removed. See
  the filtering tutorial on https://docs.qiime2.org for additional details.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                       The feature table from which features should be
                       filtered.                                    [required]
Parameters:
  --p-min-frequency INTEGER
                       The minimum total frequency that a feature must have
                       to be retained.                            [default: 0]
  --p-max-frequency INTEGER
                       The maximum total frequency that a feature can have to
                       be retained. If no value is provided this will default
                       to infinity (i.e., no maximum frequency filter will be
                       applied).                       

In [9]:
Visualization.load(f'{den_dir}/dada2_table.qzv') # we see that 5000 is a reasonable number to cutoff. But are we cutting off to many data? Now we compare the table before and after filtering it:

<visualization: Visualization uuid: f33dc630-989c-4893-83ab-cc4579390091>

In [37]:
! qiime feature-table summarize \
    --i-table $diff_abu/table_abund.qza \
    --m-sample-metadata-file $data_dir/metadata.tsv \
    --o-visualization $diff_abu/table_abund.qzv

Saved Visualization to: poop_data/Differential_abundance/table_abund.qzv


In [40]:
Visualization.load(f'{diff_abu}/table_abund.qzv') # Here we can observe that the data that we cutoff are not to many . They only decrease from 523 to 513 

<visualization: Visualization uuid: 0cdedb67-a135-4573-8b1c-f3ddbb7c25a6>

##  ANCOM: GEN_sex

In [34]:
! qiime feature-table filter-samples \
    --i-table $diff_abu/table_abund.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --p-where "[GEN_sex]='male' or [GEN_sex]='female'" \
    --o-filtered-table $diff_abu/table_abund_female_male.qza

! qiime composition add-pseudocount \
    --i-table $diff_abu/table_abund_female_male.qza \
    --o-composition-table $diff_abu/table_abund_F_M_comp.qza
    
! qiime composition ancom \
    --i-table $diff_abu/table_abund_F_M_comp.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --m-metadata-column GEN_sex \
    --p-transform-function log \
    --o-visualization $diff_abu/ancom_SEX.qzv

Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund_female_male.qza
Saved FeatureTable[Composition] to: poop_data/Differential_abundance/table_abund_F_M_comp.qza
Saved Visualization to: poop_data/Differential_abundance/ancom_SEX.qzv


In [9]:
Visualization.load(f'{diff_abu}/ancom_SEX.qzv')

<visualization: Visualization uuid: 7e88d072-7188-4257-8cb3-d5c028afb6d2>

By looking at the ANCOM Volcano we can see only one statistically significant ASV at the right corner of the plot. This is **77c2dc197e6b3dbebc4ee240c6a1c559** with a **W=36**. Then we look the same feature in the table of percentile abundances of features by group:
    **1)** in 75% of the samples in the female group, 18,25 or fewer sequences were observed and assigned to this feature
    **2)** in 75% of the samples in the male group, 1 sequences were assigned to this feature

That would suggest that this feature is more abundant in the female samples. To further investigate these results that we have done in an ASV level we are going to see the taxa down to the species and genus level.

In [3]:
! qiime taxa collapse \
    --i-table $diff_abu/table_abund.qza \
    --i-taxonomy $tox_dir/taxonomy_new.qza \
    --p-level 7 \
    --o-collapsed-table $diff_abu/table_abund_species.qza

! qiime feature-table filter-samples \
    --i-table $diff_abu/table_abund_species.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --p-where "[GEN_sex]='male' or [GEN_sex]='female'" \
    --o-filtered-table $diff_abu/table_abund_F_M_spec.qza

! qiime composition add-pseudocount \
    --i-table $diff_abu/table_abund_F_M_spec.qza \
    --o-composition-table $diff_abu/table_abund_F_M_spec_comp.qza

! qiime composition ancom \
    --i-table $diff_abu/table_abund_F_M_spec_comp.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --m-metadata-column GEN_sex \
    --o-visualization $diff_abu/ancom_GEN_sex_spec.qzv

Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund_species.qza
Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund_F_M_spec.qza
Saved FeatureTable[Composition] to: poop_data/Differential_abundance/table_abund_F_M_spec_comp.qza
Saved Visualization to: poop_data/Differential_abundance/ancom_GEN_sex_spec.qzv


In [10]:
Visualization.load(f'{diff_abu}/ancom_GEN_sex_spec.qzv') 

<visualization: Visualization uuid: 9a100427-29dc-49ef-ab57-250aadb82225>

In [4]:
! qiime taxa collapse \
    --i-table $diff_abu/table_abund.qza \
    --i-taxonomy $tox_dir/taxonomy_new.qza \
    --p-level 6 \
    --o-collapsed-table $diff_abu/table_abund_genus.qza

! qiime feature-table filter-samples \
    --i-table $diff_abu/table_abund_genus.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --p-where "[GEN_sex]='male' or [GEN_sex]='female'" \
    --o-filtered-table $diff_abu/table_abund_F_M_gen.qza

! qiime composition add-pseudocount \
    --i-table $diff_abu/table_abund_F_M_gen.qza \
    --o-composition-table $diff_abu/table_abund_F_M_gen_comp.qza

! qiime composition ancom \
    --i-table $diff_abu/table_abund_F_M_gen_comp.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --m-metadata-column GEN_sex \
    --o-visualization $diff_abu/ancom_GEN_sex_gen.qzv

Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund_genus.qza
Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund_F_M_gen.qza
Saved FeatureTable[Composition] to: poop_data/Differential_abundance/table_abund_F_M_gen_comp.qza
Saved Visualization to: poop_data/Differential_abundance/ancom_GEN_sex_gen.qzv


In [30]:
Visualization.load(f'{diff_abu}/ancom_GEN_sex_gen.qzv')  #W=11

<visualization: Visualization uuid: dbe695d7-c599-4786-b0db-59f547d879e7>

Now by looking at the ANCOM Volcano we can see an higher statistically significant difference between the two gender at the both corner of the plot. 
Also by looking to this Percentile abundances of features by group we can confirm the previous assumption that would suggest that this feature is more abundant in the female samples. 

##  ANCOM: GEN_bmi_cat

In [12]:
! qiime feature-table filter-samples \
    --i-table $diff_abu/table_abund_species.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --p-where "[GEN_bmi_cat]='Overweight' or [GEN_bmi_cat]='Normal'" \
    --o-filtered-table $diff_abu/table_abund_bmi_spec.qza

! qiime composition add-pseudocount \
    --i-table $diff_abu/table_abund_bmi_spec.qza \
    --o-composition-table $diff_abu/table_abund_bmi_spec_comp.qza

! qiime composition ancom \
    --i-table $diff_abu/table_abund_bmi_spec_comp.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --m-metadata-column GEN_bmi_cat \
    --o-visualization $diff_abu/ancom_GEN_bmi_spec.qzv

Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund_bmi_spec.qza
Saved FeatureTable[Composition] to: poop_data/Differential_abundance/table_abund_bmi_spec_comp.qza
Saved Visualization to: poop_data/Differential_abundance/ancom_GEN_bmi_spec.qzv


In [29]:
Visualization.load(f'{diff_abu}/ancom_GEN_bmi_gen.qzv')  #no significant features found

<visualization: Visualization uuid: cb4eaa37-5ea8-4374-a964-4e63e706c376>

No significant features found

We select from permanova table, where the q-value is below significance-level to find some comparison to make with ANCOM

##  ANCOM: HEA_ibd

In [18]:
! qiime feature-table filter-samples \
    --i-table $diff_abu/table_abund_species.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --p-where "[HEA_ibd]='False' or [HEA_ibd]='True'" \
    --o-filtered-table $diff_abu/table_abund_HEA_ibd_spec.qza

! qiime composition add-pseudocount \
    --i-table $diff_abu/table_abund_HEA_ibd_spec.qza \
    --o-composition-table $diff_abu/table_abund_HEA_ibd_spec_comp.qza

! qiime composition ancom \
    --i-table $diff_abu/table_abund_HEA_ibd_spec_comp.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --m-metadata-column HEA_ibd \
    --o-visualization $diff_abu/ancom_HEA_ibd_spec.qzv

Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund_HEA_ibd_spec.qza
Saved FeatureTable[Composition] to: poop_data/Differential_abundance/table_abund_HEA_ibd_spec_comp.qza
Saved Visualization to: poop_data/Differential_abundance/ancom_HEA_ibd_spec.qzv


In [28]:
Visualization.load(f'{diff_abu}/ancom_HEA_ibd_spec.qzv') #W=43

<visualization: Visualization uuid: 432e426c-a902-4688-a088-76f3e9d41be2>

##  ANCOM: HEA_cdiff

In [19]:
! qiime feature-table filter-samples \
    --i-table $diff_abu/table_abund_species.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --p-where "[HEA_cdiff]='False' or [HEA_cdiff]='True'" \
    --o-filtered-table $diff_abu/table_abund_HEA_cdiff_spec.qza

! qiime composition add-pseudocount \
    --i-table $diff_abu/table_abund_HEA_cdiff_spec.qza \
    --o-composition-table $diff_abu/table_abund_HEA_cdiff_spec_comp.qza

! qiime composition ancom \
    --i-table $diff_abu/table_abund_HEA_cdiff_spec_comp.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --m-metadata-column HEA_cdiff \
    --o-visualization $diff_abu/ancom_HEA_cdiff_spec.qzv

Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund_HEA_cdiff_spec.qza
Saved FeatureTable[Composition] to: poop_data/Differential_abundance/table_abund_HEA_cdiff_spec_comp.qza
Saved Visualization to: poop_data/Differential_abundance/ancom_HEA_cdiff_spec.qzv


In [27]:
Visualization.load(f'{diff_abu}/ancom_HEA_cdiff_spec.qzv') #No significant features found

<visualization: Visualization uuid: f248e66a-0eb7-4302-a2e4-a427610b2b40>

##  ANCOM: HEA_antibiotic_history

In [25]:
! qiime feature-table filter-samples \
    --i-table $diff_abu/table_abund_species.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --p-where "[HEA_antibiotic_history]='Year' or [HEA_antibiotic_history]='I have not taken antibiotics in the past year.'" \
    --o-filtered-table $diff_abu/table_abund_HEA_antibiotic_history_spec.qza

! qiime composition add-pseudocount \
    --i-table $diff_abu/table_abund_HEA_antibiotic_history_spec.qza \
    --o-composition-table $diff_abu/table_abund_HEA_antibiotic_history_spec_comp.qza

! qiime composition ancom \
    --i-table $diff_abu/table_abund_HEA_antibiotic_history_spec_comp.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --m-metadata-column HEA_cdiff \
    --o-visualization $diff_abu/ancom_HEA_antibiotic_history_spec.qzv

Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund_HEA_antibiotic_history_spec.qza
Saved FeatureTable[Composition] to: poop_data/Differential_abundance/table_abund_HEA_antibiotic_history_spec_comp.qza
Saved Visualization to: poop_data/Differential_abundance/ancom_HEA_antibiotic_history_spec.qzv


In [26]:
Visualization.load(f'{diff_abu}/ancom_HEA_antibiotic_history_spec.qzv')  #No significant features found

<visualization: Visualization uuid: e3801244-6b42-4de4-aac4-24e5fc3787e5>

from Nina folder

##  ANCOM: HEA_migraine

In [21]:
! qiime feature-table filter-samples \
    --i-table $diff_abu/table_abund_species.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --p-where "[HEA_migraine]='False' or [HEA_migraine]='True'" \
    --o-filtered-table $diff_abu/table_abund_HEA_migraine_spec.qza

! qiime composition add-pseudocount \
    --i-table $diff_abu/table_abund_HEA_migraine_spec.qza \
    --o-composition-table $diff_abu/table_abund_HEA_migraine_spec_comp.qza

! qiime composition ancom \
    --i-table $diff_abu/table_abund_HEA_migraine_spec_comp.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --m-metadata-column HEA_migraine \
    --o-visualization $diff_abu/ancom_HEA_migraine_spec.qzv

Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund_HEA_migraine_spec.qza
Saved FeatureTable[Composition] to: poop_data/Differential_abundance/table_abund_HEA_migraine_spec_comp.qza
Saved Visualization to: poop_data/Differential_abundance/ancom_HEA_migraine_spec.qzv


In [24]:
Visualization.load(f'{diff_abu}/ancom_HEA_migraine_spec.qzv') #No significant features found

<visualization: Visualization uuid: 8a68931e-fd3f-4461-ac92-ab3dc098ae90>

##  ANCOM: HEA_mental_illness

In [22]:
! qiime feature-table filter-samples \
    --i-table $diff_abu/table_abund_species.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --p-where "[HEA_mental_illness]='False' or [HEA_mental_illness]='True'" \
    --o-filtered-table $diff_abu/table_abund_HEA_mental_illness_spec.qza

! qiime composition add-pseudocount \
    --i-table $diff_abu/table_abund_HEA_mental_illness_spec.qza \
    --o-composition-table $diff_abu/table_abund_HEA_mental_illness_spec_comp.qza

! qiime composition ancom \
    --i-table $diff_abu/table_abund_HEA_mental_illness_spec_comp.qza \
    --m-metadata-file $data_dir/metadata.tsv \
    --m-metadata-column HEA_mental_illness \
    --o-visualization $diff_abu/ancom_HEA_mental_illness_spec.qzv

Saved FeatureTable[Frequency] to: poop_data/Differential_abundance/table_abund_HEA_mental_illness_spec.qza
Saved FeatureTable[Composition] to: poop_data/Differential_abundance/table_abund_HEA_mental_illness_spec_comp.qza
Saved Visualization to: poop_data/Differential_abundance/ancom_HEA_mental_illness_spec.qzv


In [23]:
Visualization.load(f'{diff_abu}/ancom_HEA_mental_illness_spec.qzv') #No significant features found

<visualization: Visualization uuid: fa3cb52e-5918-4e75-b700-ebdf6c8a8291>

##  ANCOM: HEA_HEA_sibo